In [ ]:
import pandas as pd

In [ ]:
%%capture
!pip install gdown

In [ ]:
!gdown "https://drive.google.com/uc?id=1i-sTZzhYwf_ShhTCFNGyMNugaVIkMix6"


Downloading...
From: https://drive.google.com/uc?id=1i-sTZzhYwf_ShhTCFNGyMNugaVIkMix6
To: /content/Eval.zip
100% 1.29M/1.29M [00:00<00:00, 192MB/s]


In [ ]:
! unzip /content/Eval.zip

Archive:  /content/Eval.zip
   creating: Eval/
   creating: Eval/model_base_7/
  inflating: Eval/model_base_7/base_model_outputs.csv  
  inflating: Eval/model_base_7/metrics.json  
  inflating: Eval/model_base_7/desktop.ini  
   creating: Eval/model_base_1/
  inflating: Eval/model_base_1/metric base model 1.json  
  inflating: Eval/model_base_1/base_model_outputs.csv  
  inflating: Eval/model_base_1/desktop.ini  
   creating: Eval/model_ft_1/
  inflating: Eval/model_ft_1/fine tune base model.json  
  inflating: Eval/model_ft_1/fine_tuned_outputs.csv  
  inflating: Eval/model_ft_1/desktop.ini  
   creating: Eval/model_ft_7/
  inflating: Eval/model_ft_7/fine_tuned_outputs.csv  
  inflating: Eval/model_ft_7/metrics.json  
  inflating: Eval/model_ft_7/desktop.ini  
   creating: Eval/model_base_8/
  inflating: Eval/model_base_8/base_model_outputs.csv  
  inflating: Eval/model_base_8/metrics.json  
  inflating: Eval/model_base_8/desktop.ini  
   creating: Eval/model_base_5/
   creating: Eval

In [ ]:
# prompt: import bert score, bleu, rougr
%%capture
!pip install bert-score
!pip install evaluate
!pip install rouge_score

In [ ]:
import bert_score
from evaluate import load
import pandas as pd

In [ ]:
import evaluate

In [ ]:
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import evaluate

bleu = evaluate.load("bleu")

predictions = ["the cat is on the mat"]
references = [["there is a cat on the mat"]]

# BLEU-1
result_bleu1 = bleu.compute(predictions=predictions, references=references, max_order=1)
print(f"BLEU-1: {result_bleu1['bleu']:.4f}")

# BLEU-2
result_bleu2 = bleu.compute(predictions=predictions, references=references, max_order=2)
print(f"BLEU-2: {result_bleu2['bleu']:.4f}")

# BLEU-3
result_bleu3 = bleu.compute(predictions=predictions, references=references, max_order=3)
print(f"BLEU-3: {result_bleu3['bleu']:.4f}")

# BLEU-4 (default)
result_bleu4 = bleu.compute(predictions=predictions, references=references)
print(f"BLEU-4: {result_bleu4['bleu']:.4f}")


BLEU-1: 0.7054
BLEU-2: 0.4887
BLEU-3: 0.3697
BLEU-4: 0.0000


In [ ]:
pd.options.display.max_colwidth = 500

Function Compute Bleu score


In [ ]:
import evaluate
import pandas as pd

# Load evaluator sekali saja
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

def compute_bleu_per_row(predictions, references):
    """
    Menghitung BLEU 1-4 untuk setiap pasangan prediksi dan referensi.
    Return: list of dict, tiap dict {'bleu1':..., 'bleu2':..., 'bleu3':..., 'bleu4':...}
    """
    all_scores = []
    for pred, ref in zip(predictions, references):
        scores = {}
        # ref harus list of references, meskipun 1 referensi
        ref_list = [ref] if isinstance(ref, str) else ref
        for n in range(1, 5):
            score = bleu.compute(predictions=[pred], references=[ref_list], max_order=n, smooth=True)
            scores[f'bleu{n}'] = score['bleu']
        all_scores.append(scores)
    return all_scores

Function Compoute Rouge Score

In [ ]:
def compute_rouge_per_row(predictions, references):
    rouge = evaluate.load("rouge")
    scores = []
    for pred, ref in zip(predictions, references):
        score = rouge.compute(predictions=[pred], references=[ref])
        # Ambil langsung fmeasure-nya (hasil evaluate adalah dict float)
        scores.append({
            "rouge1": score["rouge1"],
            "rouge2": score["rouge2"],
            "rougeL": score["rougeL"],
            "rougeLSum": score["rougeLsum"]
        })
    return scores


Bert score multiligual bert

In [ ]:
import evaluate

def compute_bertscore_per_row(predictions, references):
    """
    Menghitung BERTScore per kalimat untuk Bahasa Indonesia menggunakan library evaluate.

    Parameters:
    - predictions: List[str], list kalimat yang dihasilkan model.
    - references: List[List[str]], list referensi yang diharapkan (nested list, satu referensi per kalimat).

    Returns:
    - List of Dicts berisi precision, recall, dan F1 untuk setiap kalimat.
    """
    # Muat evaluator
    bertscore = evaluate.load("bertscore")

    # Pastikan referensi adalah list of strings (flatten)
    flat_references = [ref[0] for ref in references]

    # Hitung skor
    results = bertscore.compute(
        predictions=predictions,
        references=flat_references,
        lang="id"  # Bahasa Indonesia
    )

    # Gabungkan hasil per kalimat
    scores_per_row = []
    for i in range(len(predictions)):
        scores_per_row.append({
            "prediction": predictions[i],
            "reference": flat_references[i],
            "bert_precision": results["precision"][i],
            "bert_recall": results["recall"][i],
            "bert_f1": results["f1"][i],
        })

    return scores_per_row

# Contoh penggunaan
predictions = ["kucing itu duduk di atas tikar"]
references = [["seekor kucing duduk di tikar"]]

scores = compute_bertscore_per_row(predictions, references)

print("BERTScore per row:")
for i, score in enumerate(scores):
    print(f"Row {i+1}:")
    print(f"  Prediction: {score['prediction']}")
    print(f"  Reference : {score['reference']}")
    print(f"  Precision : {score['bert_precision']:.4f}")
    print(f"  Recall    : {score['bert_recall']:.4f}")
    print(f"  F1        : {score['bert_f1']:.4f}")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_REMOVED_BY_SCRIPT' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_REMOVED_BY_SCRIPT]` or `pip install hf_REMOVED_BY_SCRIPT`


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

BERTScore per row:
Row 1:
  Prediction: kucing itu duduk di atas tikar
  Reference : seekor kucing duduk di tikar
  Precision : 0.7809
  Recall    : 0.7824
  F1        : 0.7817


# Model 3

## Base Model 3

In [ ]:
base_3 = pd.read_csv('/content/Eval/model_base_3/content/eval_basemodel_20250525_025234/base_model_outputs.csv')
base_3.sample(1)

,instruction,input,reference,generated,bert_precision,bert_recall,bert_f1
19,Perut sebelah kiri mengeras,Perut sebelah kiri mengeras. Perut kiri saya mengeras Dan terkadang Teresa nyeri Di punggung apakah berpengaruh jika menahan lapar.,"yakni lambung, usus, kelenjar pankreas, dan ginjal. jika terdapat rasa seperti mengeras pada perut anda sebelah kiri kemungkinan terdapat gangguan pada organ-organ tersebut. berikut beberapa gangguan pada perut kiri terkait organ-organ tersebut adalah : terjadinya cedera yang mengenai perut sebelah kiri terdapat infeksi ginjal adanya batu pada ginjal gangguan pada lambung seperti tukak lambung , gastritis , penyakit asam lambung tersumbatnya tinja didalam usus dan mengeras rasa nyeri yang te...","Berdasarkan perintah ""Perut sebelah kiri mengeras,"" arti dari teks adalah ""Perut kiri mengeras, dan Teresa merasa sakit di punggung, apakah hal ini mengganggu apabila tidak makan?"" Jawaban:\n\nSebelah kiri perut merupakan lokasi di mana gandum dan organ-organ kecil berada. Gandum tersebut boleh meneruskan fungsi perut sejak itu. Kadang-kadang, kegagalan atau masalah dalam perut kiri dapat menyebabkan sakit di punggung. Namun, terus menahan sakit ini selama waktu panjang dapat mengganggu kesi...",0.66675,0.632824,0.649344


In [ ]:
base_3.isna().sum()

,0
instruction,0
input,0
reference,0
generated,0
bert_precision,0
bert_recall,0
bert_f1,0


In [ ]:
# Buang baris yang memiliki nan pada kolom generated atau reference
df_clean = base_3.dropna(subset=['generated', 'reference']).copy()

preds = df_clean['generated'].tolist()
refs = df_clean['reference'].tolist()

# Pastikan references berbentuk list of list untuk BLEU
refs_for_bleu = [[ref] for ref in refs]

bleu_scores = compute_bleu_per_row(preds, refs_for_bleu)
rouge_scores = compute_rouge_per_row(preds, refs)

# Buat dict untuk dataframe baru
data_new = {}

for n in range(1, 5):
    data_new[f'bleu{n}'] = [score[f'bleu{n}'] for score in bleu_scores]

data_new['rouge1'] = [score['rouge1'] for score in rouge_scores]
data_new['rouge2'] = [score['rouge2'] for score in rouge_scores]
data_new['rougeL'] = [score['rougeL'] for score in rouge_scores]
data_new['rougeLSum'] = [score['rougeLSum'] for score in rouge_scores]

In [ ]:
bertscore_eval_scores = compute_bertscore_per_row(preds, refs_for_bleu)

In [ ]:
df_bert_score = pd.DataFrame(bertscore_eval_scores)
df_bert_score

,prediction,reference,bert_precision,bert_recall,bert_f1
0,"Based on your description, it seems that you're experiencing discomfort in the middle of your back, and it's been troubling you since last night. You mentioned that it doesn't hurt or feel painful, but rather, it feels uncomfortable, like there's a lump or a gnawing sensation, and it's possible that you might have an indigestion issue since you mentioned having a bloated left side. It would be a good idea for you to consult a doctor to rule out any potential health concerns. In the meantime,...","kondisi yang anda alami saat ini dengan keluhan utama berupa rasa tidak nyaman pada daerah dada bagian tengah atau dapat disebut sebagai ulu hati, paling banyak disebabkan oleh karena gangguan pada organ lambung. berikut ini beberapa penyakit yang dapat mengganggu kesehatan organ lambung, yaitu : gastritis=== peradangan pada dinding dalam lambung, tukak peptik=== luka yang terdapat pada dinding dalam lambung, penyakit asam lambung / gerd. sedangkan selain oleh karena gangguan pada organ lamb...",0.626971,0.601298,0.613866
1,"Based on the information provided, it appears that you have been experiencing symptoms of gastrointestinal distress for the past two weeks, including abdominal pain, weight loss, loss of appetite, and nausea with vomiting. These symptoms are consistent with gastroesophageal reflux disease (GERD), also known as acid reflux. Despite taking antacids, such as omeprazole and ranitidin, your symptoms have not improved. Given the severity and duration of your symptoms, I would recommend that you co...","usus halus ataupun kerongkongan yang dapat disebabkan kondisi berikut : faktor psikis, yakni kondisi stres emosional akibat pengunaan obat antinyeri non steroid (nsaid) akibat perlukaan pada lambung (tukak lambung/peptik) infeksi bakteri helicobacter pylori (gastritis) dsb sakit maag yang kronis atau berlangsung berat dapat menyebabkan perlukaan pada lambung dan sebaliknya. pada perlukaan yang berat dapat terjadi perforasi (pecahnya) jaringan lambung sehingga menimbulkan gejala batuk/muntah ...",0.629694,0.596360,0.612574
2,"Masalah konstipasi yang anda alami dapat disebabkan oleh beberapa faktor, seperti kekurangan minyak lemak atau air dalam makanan anda, tidak makan cukup atau tidak bergerak cukup. Untuk mengatasi masalah ini, anda dapat mencoba beberapa hal seperti:\n\n1. Minum air secukupnya setiap hari untuk memastikan anda terhindar dari dehidrasi.\n2. Makanan yang mengandung fiber tinggi seperti sayuran, buah-buahan, dan jagung.\n3. Bergerak secara teratur setiap hari, seperti menjalani atau berolahraga....",konstipasi adalah keadaan tidak dapat buang air besar secara teratur atau tidak dapat buang air besar sama sekali. konstipasi dapat disebabkan oleh: kurang mimun kurang konsumsi serat kurang aktif bergerak dan berolahraga kelebihan atau kekurangan berat badan kebiasaan menunda buang air besar cemas atau depresi penyumbatan usus gangguan saraf usus efek samping obat-obatan tertentu dsb. sebaiknya anda berkonsultasi dengan dokter di mana dokter akan menganalisis gejala dan melakukan pemeriksaa...,0.674171,0.653103,0.663469
3,"I'm sorry to hear that your wife is experiencing abdominal pain, bloating, and excessive gas. This could be due to a number of reasons, including irritable bowel syndrome (IBS), food intolerances, or gastrointestinal infections. To determine the cause, it would be best for her to consult with a healthcare professional who can perform a thorough examination and possibly order some tests. In the meantime, she may find relief by avoiding certain foods that trigger symptoms, practicing stress re...","penyebab sakit perut, mual dan bersendawa mungkin disebabkan oleh sin, yang gejala-gejalanya adalah: sering bersendawa tidak nyaman di perut, kembung mual penurunan nafsu makan dada terasa seperti terbakar penyebab dispepsia ini diantaranya: refluks asam lambung (gerd) ulkus peptikum intoleransi laktosa batu empedu dan radang empedu depres

In [ ]:
df_scores = pd.DataFrame(data_new)
df_scores = df_scores * 100
df_scores = pd.concat([df_bert_score, df_scores], axis=1)
df_scores.head(2)

,prediction,reference,bert_precision,bert_recall,bert_f1,bleu1,bleu2,bleu3,bleu4,rouge1,rouge2,rougeL,rougeLSum
0,"Based on your description, it seems that you're experiencing discomfort in the middle of your back, and it's been troubling you since last night. You mentioned that it doesn't hurt or feel painful, but rather, it feels uncomfortable, like there's a lump or a gnawing sensation, and it's possible that you might have an indigestion issue since you mentioned having a bloated left side. It would be a good idea for you to consult a doctor to rule out any potential health concerns. In the meantime,...","kondisi yang anda alami saat ini dengan keluhan utama berupa rasa tidak nyaman pada daerah dada bagian tengah atau dapat disebut sebagai ulu hati, paling banyak disebabkan oleh karena gangguan pada organ lambung. berikut ini beberapa penyakit yang dapat mengganggu kesehatan organ lambung, yaitu : gastritis=== peradangan pada dinding dalam lambung, tukak peptik=== luka yang terdapat pada dinding dalam lambung, penyakit asam lambung / gerd. sedangkan selain oleh karena gangguan pada organ lamb...",0.626971,0.601298,0.613866,4.074966,1.181358,0.784109,0.640198,0.0,0.0,0.0,0.0
1,"Based on the information provided, it appears that you have been experiencing symptoms of gastrointestinal distress for the past two weeks, including abdominal pain, weight loss, loss of appetite, and nausea with vomiting. These symptoms are consistent with gastroesophageal reflux disease (GERD), also known as acid reflux. Despite taking antacids, such as omeprazole and ranitidin, your symptoms have not improved. Given the severity and duration of your symptoms, I would recommend that you co...","usus halus ataupun kerongkongan yang dapat disebabkan kondisi berikut : faktor psikis, yakni kondisi stres emosional akibat pengunaan obat antinyeri non steroid (nsaid) akibat perlukaan pada lambung (tukak lambung/peptik) infeksi bakteri helicobacter pylori (gastritis) dsb sakit maag yang kronis atau berlangsung berat dapat menyebabkan perlukaan pada lambung dan sebaliknya. pada perlukaan yang berat dapat terjadi perforasi (pecahnya) jaringan lambung sehingga menimbulkan gejala batuk/muntah ...",0.629694,0.596360,0.612574,7.122213,1.521893,0.911235,0.705908,0.0,0.0,0.0,0.0


In [ ]:
# Calculate the mean of the numerical columns only
numerical_cols = ['bert_precision', 'bert_recall', 'bert_f1', 'bleu1', 'bleu2', 'bleu3', 'bleu4', 'rouge1', 'rouge2', 'rougeL', 'rougeLSum']
mean_scores = df_scores[numerical_cols].mean()

print("\nRata-rata Skor:")
mean_scores


Rata-rata Skor:


,0
bert_precision,0.651668
bert_recall,0.631243
bert_f1,0.640990
bleu1,12.960688
bleu2,4.302881
bleu3,2.145769
bleu4,1.413015
rouge1,13.847933
rouge2,1.864717
rougeL,7.207788


In [ ]:
import os
os.makedirs('Hasil Eval revisi')
mean_scores.to_csv('Hasil Eval revisi/eval_basemodel_3_mean.csv')

In [ ]:
df_scores = pd.concat([base_3[['input']], df_scores], axis=1)
df_scores.head(2)

,input,prediction,reference,bert_precision,bert_recall,bert_f1,bleu1,bleu2,bleu3,bleu4,rouge1,rouge2,rougeL,rougeLSum
0,"dada bagian tengah terasa tidak nyaman. saya mengalami ketidak nyamanan di dada atas tengah saya , dikatakan sesak, tidak!, tidak nyeri amat, semuanya lancar,,,, tapi agak terasa enggak enak seperti ada ganjalan atau seperti masuk angin, Tapi hal tersebut baru terasa sejak malam tadi, Saya kira masuk angin karena perut saya bagian kiri kembung, dan saya pernah sakit asam lambung","Based on your description, it seems that you're experiencing discomfort in the middle of your back, and it's been troubling you since last night. You mentioned that it doesn't hurt or feel painful, but rather, it feels uncomfortable, like there's a lump or a gnawing sensation, and it's possible that you might have an indigestion issue since you mentioned having a bloated left side. It would be a good idea for you to consult a doctor to rule out any potential health concerns. In the meantime,...","kondisi yang anda alami saat ini dengan keluhan utama berupa rasa tidak nyaman pada daerah dada bagian tengah atau dapat disebut sebagai ulu hati, paling banyak disebabkan oleh karena gangguan pada organ lambung. berikut ini beberapa penyakit yang dapat mengganggu kesehatan organ lambung, yaitu : gastritis=== peradangan pada dinding dalam lambung, tukak peptik=== luka yang terdapat pada dinding dalam lambung, penyakit asam lambung / gerd. sedangkan selain oleh karena gangguan pada organ lamb...",0.626971,0.601298,0.613866,4.074966,1.181358,0.784109,0.640198,0.0,0.0,0.0,0.0
1,"Solusi atasi sakit maag yang tak kunjung sembuh. Saya memiliki keluhan selama 2 minggu belakangan ada masalah di pencernaan. Saya sudah berobat klinik 2 kali tapi diagnosanya sama yaitu saya menderita maag. Oleh dokter yang menangani, saya hanya diberi antasida, omeprazole, dan ranitidin. Saya sudah habiskan obatnya, pola makan pun saya jaga dengan tidak mengkonsumsi makanan pedas, asam, dan kafein namun sampai saat ini tidak ada perubahan. Selama 2 minggu saya sakit berat badan saya turun 1...","Based on the information provided, it appears that you have been experiencing symptoms of gastrointestinal distress for the past two weeks, including abdominal pain, weight loss, loss of appetite, and nausea with vomiting. These symptoms are consistent with gastroesophageal reflux disease (GERD), also known as acid reflux. Despite taking antacids, such as omeprazole and ranitidin, your symptoms have not improved. Given the severity and duration of your symptoms, I would recommend that you co...","usus halus ataupun kerongkongan yang dapat disebabkan kondisi berikut : faktor psikis, yakni kondisi stres emosional akibat pengunaan obat antinyeri non steroid (nsaid) akibat perlukaan pada lambung (tukak lambung/peptik) infeksi bakteri helicobacter pylori (gastritis) dsb sakit maag yang kronis atau berlangsung berat dapat menyebabkan perlukaan pada lambung dan sebaliknya. pada perlukaan yang berat dapat terjadi perforasi (pecahnya) jaringan lambung sehingga menimbulkan gejala batuk/muntah ...",0.629694,0.596360,0.612574,7.122213,1.521893,0.911235,0.705908,0.0,0.0,0.0,0.0


In [ ]:
df_scores.to_csv('Hasil Eval revisi/eval_basemodel_3.csv')

## Fine Tune Model 1

In [ ]:
def evaluate_generation_scores(data):
    # Buang baris yang memiliki NaN pada kolom generated atau reference
    df_clean = data.dropna(subset=['generated', 'reference']).copy()

    preds = df_clean['generated'].tolist()
    refs = df_clean['reference'].tolist()

    # Pastikan references berbentuk list of list untuk BLEU
    refs_for_bleu = [[ref] for ref in refs]

    # Hitung BLEU dan ROUGE per baris
    bleu_scores = compute_bleu_per_row(preds, refs_for_bleu)
    rouge_scores = compute_rouge_per_row(preds, refs)

    # Hitung BERTScore per baris
    bert_scores = compute_bertscore_per_row(preds, refs_for_bleu)

    # Siapkan dictionary untuk skor
    data_new = {}

    for n in range(1, 5):
        data_new[f'bleu{n}'] = [score[f'bleu{n}'] for score in bleu_scores]

    data_new['rouge1'] = [score['rouge1'] for score in rouge_scores]
    data_new['rouge2'] = [score['rouge2'] for score in rouge_scores]
    data_new['rougeL'] = [score['rougeL'] for score in rouge_scores]
    data_new['rougeLSum'] = [score['rougeLSum'] for score in rouge_scores]

    # Tambahkan BERTScore per row
    data_new['bert_precision'] = [score['bert_precision'] for score in bert_scores]
    data_new['bert_recall'] = [score['bert_recall'] for score in bert_scores]
    data_new['bert_f1'] = [score['bert_f1'] for score in bert_scores]

    # Konversi ke DataFrame dan skalakan ke persen
    df_scores = pd.DataFrame(data_new)
    df_scores = df_scores * 100

    # Tambahkan kolom input, reference, dan generated
    df_scores['input'] = df_clean['input'].values
    df_scores['reference'] = df_clean['reference'].values
    df_scores['generated'] = df_clean['generated'].values

    # Hitung rata-rata skor
    mean_scores = df_scores.drop(columns=['input', 'reference', 'generated']).mean()

    return df_scores, mean_scores


In [ ]:
ft_model_3 = pd.read_csv('/content/Eval/model_ft_3/content/eval_basemodel_20250525_035241/fine_tuned_outputs.csv')
ft_model_3.sample(1)

,instruction,input,reference,generated,bert_precision,bert_recall,bert_f1
25,Perut sering berbunyi disertai buang angin dan sendawa,"Perut sering berbunyi disertai buang angin dan sendawa. 13 menjelang 14 saya mempunyai penyakit maag, saya jarang sarapan,dan sudah 4 hari saya mual setiap waktu,air liur lebih banyak di produksi,dan tidak ada selera makan,terus perut saya sering bunyi seperti sedang mencerna setelah perut saya bunyi saya selalu buang angin saya sering bersendawa,","seperti sakit maag , luka (tukak) pada dinding lambung, penyakit asam lambung , terlalu banyak makan makanan berlemak&gorengan, merokok/terlalu banyak menghirup asap rokok, makan terlalu cepat, kelelahan fisik, stres emosional, iritasi lambung karena mengkonsumsi coklat, minuman berkafein dan beralkohol. mual , disertai produksi air liur yang meningkat merupakan tanda bahwa produksi asam lambung berlebihan. hal ini juga menyebabkan peningkatan produksi gas pencernaan sehingga muncul keluhan ...","penyakit asam lambung merupakan kondisi naiknya asam lambung ke kerongkongan. pada penyakit asam lambung, terjadi kenaikan asam lambung ke kerongkongan yang menimbulkan gejala seperti rasa panas pada dada, rasa tidak nyaman di dada, rasa pahit di mulut, rasa panas di tenggorokan, batuk kering, mual, muntah, dada terasa berat, dulu. untuk memastikan penyebab keluhan anda, diperlukan pemeriksaan langsung oleh dokter. mungkin diperlukan pemeriksaan darah, usg, atau endoskopi untuk menunjang dia...",0.708216,0.669149,0.688128


In [ ]:
df_ft, mean_ft = evaluate_generation_scores(ft_model_3)

In [ ]:
df_ft.head(2)

,bleu1,bleu2,bleu3,bleu4,rouge1,rouge2,rougeL,rougeLSum,bert_precision,bert_recall,bert_f1,input,reference,generated
0,30.722892,14.289874,7.187282,3.875985,30.501931,6.976744,16.988417,16.988417,72.311115,70.999652,71.649384,"Solusi atasi sakit maag yang tak kunjung sembuh. Saya memiliki keluhan selama 2 minggu belakangan ada masalah di pencernaan. Saya sudah berobat klinik 2 kali tapi diagnosanya sama yaitu saya menderita maag. Oleh dokter yang menangani, saya hanya diberi antasida, omeprazole, dan ranitidin. Saya sudah habiskan obatnya, pola makan pun saya jaga dengan tidak mengkonsumsi makanan pedas, asam, dan kafein namun sampai saat ini tidak ada perubahan. Selama 2 minggu saya sakit berat badan saya turun 1...","usus halus ataupun kerongkongan yang dapat disebabkan kondisi berikut : faktor psikis, yakni kondisi stres emosional akibat pengunaan obat antinyeri non steroid (nsaid) akibat perlukaan pada lambung (tukak lambung/peptik) infeksi bakteri helicobacter pylori (gastritis) dsb sakit maag yang kronis atau berlangsung berat dapat menyebabkan perlukaan pada lambung dan sebaliknya. pada perlukaan yang berat dapat terjadi perforasi (pecahnya) jaringan lambung sehingga menimbulkan gejala batuk/muntah ...","mual, muntah, dan sakit perut bagian atas. berdasarkan penyebabnya, sakit maag dibagi menjadi: dispepsia fungsional, yaitu sakit maag yang tidak diketahui penyebabnya. dispepsia fungsional paling sering terjadi karena faktor psikologis seperti stres, kecemasan, cemas, panik, atau depresi. dispepsia fungsional dapat disebabkan oleh: perubahan hormonal, misalnya saat menjelang menstruasi, kehamilan, dan menopause kebiasaan makan yang tidak teratur, misalnya terlambat makan efek samping obat, m..."
1,51.912568,39.607908,31.770861,25.833819,58.360656,33.663366,45.901639,45.901639,75.655496,77.175951,76.408154,"solusi untuk mengatasi konstipasi. dok kalo bab feses saya enggak keras, cair tapi usus saya enggak mau untuk ngeluarinnya. kenapa ya dok?",konstipasi adalah keadaan tidak dapat buang air besar secara teratur atau tidak dapat buang air besar sama sekali. konstipasi dapat disebabkan oleh: kurang mimun kurang konsumsi serat kurang aktif bergerak dan berolahraga kelebihan atau kekurangan berat badan kebiasaan menunda buang air besar cemas atau depresi penyumbatan usus gangguan saraf usus efek samping obat-obatan tertentu dsb. sebaiknya anda berkonsultasi dengan dokter di mana dokter akan menganalisis gejala dan melakukan pemeriksaa...,"konstipasi atau sulit buang air besar merupakan kondisi dimana terjadi kesulitan atau tidak dapat buang air besar secara teratur atau tidak dapat sama sekali.konstipasi dapat disebabkan oleh beberapa hal, antara lain: pola makan yang buruk, seperti kurang minum, kurang serat, kurang aktivitas fisik kebiasaan mengabaikan keinginan buang air besar sering menunda buang air besar efek samping obat-obatan tertentu kondisi medis tertentu, seperti penyumbatan usus, penyempitan usus, penyakit parkin..."


In [ ]:
mean_ft

,0
bleu1,27.092743
bleu2,13.047758
bleu3,7.174046
bleu4,4.485455
rouge1,31.162396
rouge2,6.961255
rougeL,16.031524
rougeLSum,16.031524
bert_precision,69.342242
bert_recall,69.161179


In [ ]:
df_ft.to_csv('Hasil Eval revisi/eval_ft_model_3.csv')

In [ ]:
mean_ft.to_csv('Hasil Eval revisi/eval_ft_model_3_mean.csv')

In [ ]:
!zip -r /content/Hasil_Eval_revisi-3.zip /content/Hasil\ Eval\ revisi

  adding: content/Hasil Eval revisi/ (stored 0%)
  adding: content/Hasil Eval revisi/eval_ft_model_3.csv (deflated 71%)
  adding: content/Hasil Eval revisi/eval_basemodel_3.csv (deflated 66%)
  adding: content/Hasil Eval revisi/eval_basemodel_3_mean.csv (deflated 38%)
  adding: content/Hasil Eval revisi/eval_ft_model_3_mean.csv (deflated 40%)
